# Image Classification with Tensorflow

This tutorial demonstrates the steps required to prepare and deploy a trained Tensorflow model for FPGA acceleration using Xilinx MLSuite:  
1. **Quantize the model** - The quantizer will generate scaling parameters for quantizing floats INT8. This is required, because FPGAs will take advantage of Fixed Point Precision, to achieve more parallelization at lower power. 
2. **Subgraph Cutting and Compilation** - In this step, the original graph is cut, compiled and a custom FPGA accelerated python layer is inserted to be used for Inference. 
4. **Classification** - In this step, the modified Tensorflow model from the previous step are run on the FPGA to perform inference on an input image.
    
## Prerequisite Files 
1. **Model files** - This notebook requires that model files are located in  
  `/opt/models/tensorflow`
2. **Image files** - This notebook requires ilsvrc2012 image files are downloaded in  
  `HOME/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/`
  
## Setup (Before Running Notebook)
This notebook should be run inside a tensorflow docker container.

Download the models to "/opt/models/tensorflow":
```
$ python /opt/ml-suite/examples/tensorflow/getModels.py
```

Download 500 calibration images into "~/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/":
```
$ python -m ck pull repo:ck-env
$ python -m ck install package:imagenet-2012-val-min
$ python -m ck install package:imagenet-2012-aux
$ head -n 500 ~/CK-TOOLS/dataset-imagenet-ilsvrc2012-aux/val.txt > ~/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/val.txt
```

### Step 1. Import required packages

In [ ]:
from __future__ import print_function

import os
import re

from IPython.display import Image as display
from ipywidgets import interact

import tensorflow as tf
import numpy as np
import cv2

# Environment Variables (obtained by running "source overlaybins/setup.sh")
HOME             = os.getenv('HOME','/home/mluser/')
MLSUITE_ROOT     = os.getenv('MLSUITE_ROOT',os.getcwd()+'/../')
MLSUITE_PLATFORM = os.getenv('MLSUITE_PLATFORM','alveo-u200')
XCLBIN           = MLSUITE_ROOT + '/overlaybins/' + MLSUITE_PLATFORM + '/overlay_4.xclbin'

MODELDIR   = "/opt/models/tensorflow/"
IMAGEDIR   = HOME + "/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/"
IMAGELIST  = HOME + "/CK-TOOLS/dataset-imagenet-ilsvrc2012-val-min/val.txt"
LABELSLIST = HOME + "/CK-TOOLS/dataset-imagenet-ilsvrc2012-aux/synset_words.txt"

print("Running w/ HOME: %s" % HOME)
print("Running w/ MLSUITE_ROOT: %s" % MLSUITE_ROOT)
print("Running w/ XCLBIN: %s" % XCLBIN)
print("Running w/ MLSUITE_PLATFORM: %s" % MLSUITE_PLATFORM)

# moving to MLSUITE_ROOT directory
os.chdir(MLSUITE_ROOT)

# Bring in ml-suite Quantizer, Compiler, and Partitioner
from xfdnn.rt.xdnn_rt_tf import TFxdnnRT as xdnnRT
from xfdnn.rt.xdnn_util import make_list
from xfdnn.rt.xdnn_io import default_xdnn_arg_parser

### Step 2. Choose a model
Choose a model using the drop down, or select custom, and enter your own.

In [ ]:
@interact(MODEL=['resnet50','inception_v1','custom'])
def selectModel(MODEL):
    global protoBuffer, quantInfo, inputNode, outputNode, inputShape, means

    default_protoBuffer = {'resnet50': 'resnet50_baseline.pb', 'inception_v1': 'inception_v1_baseline.pb', 'pedestrian_attribute': 'pedestrian_attributes_recognition_quantizations.pb'}
    default_quantInfo   = {'resnet50': 'fix_info_resnet50_baseline.txt', 'inception_v1': 'fix_info_inception_v1_baseline.txt', 'pedestrian_attribute': 'fix_info_pedestrian_attributes_recognition_quantizations.txt'}
    default_inputNode   = {'resnet50': 'data', 'inception_v1': 'data', 'pedestrian_attribute': 'data'}
    default_outputNode  = {'resnet50': 'prob', 'inception_v1': 'loss3_loss3', 'pedestrian_attribute': 'pred_upper,pred_lower,pred_gender,pred_hat,pred_bag,pred_handbag,pred_backpack'}
    default_inputShape  = {'resnet50': '224,224', 'inception_v1': '224,224', 'pedestrian_attribute': '224,128'}
    default_means       = {'resnet50': '104,107,123', 'inception_v1': '104,107,123', 'pedestrian_attribute': '104,107,123'}

    if MODEL == "custom":
        protoBuffer = None
        quantInfo   = None
        inputNode   = None
        outputNode  = None
        inputShape  = None
        means       = None
    else:
        protoBuffer = MODELDIR + default_protoBuffer[MODEL]
        quantInfo   = MODELDIR + default_quantInfo[MODEL]
        inputNode   = default_inputNode[MODEL]
        outputNode  = default_outputNode[MODEL]
        inputShape  = default_inputShape[MODEL]
        means       = default_means[MODEL]

In [ ]:
if not protoBuffer:
    @interact(PROTOBUFFER="Provide the path to your protobuffer")
    def selectPrototxt(PROTOBUFFER):
        global protoBuffer
        protoBuffer = PROTOBUFFER

if not quantInfo:
    @interact(QUANTINFO="Provide the path to your quantization file")
    def selectCaffemodel(QUANTINFO):
        global quantInfo
        quantInfo = QUANTINFO

if not inputNode:
    @interact(INPUTNODE="Provide the input node(s) (comma separated string with no spaces)")
    def selectCaffemodel(INPUTNODE):
        global inputNode
        inputNode = INPUTNODE

if not outputNode:
    @interact(OUTPUTNODE="Provide the output node(s) (comma separated string with no spaces)")
    def selectCaffemodel(INPUTNODE):
        global outputNode
        outputNode = OUTPUTNODE

if not inputShape:
    @interact(INPUTSHAPE="Provide the input shapes (comma separated string with no spaces)")
    def selectCaffemodel(INPUTSHAPE):
        global inputShape
        inputShape = INPUTSHAPE

if not means:
    @interact(MEANS="Provide the means (comma separated string with no spaces)")
    def selectCaffemodel(MEANS):
        global means
        means = MEANS

In [ ]:
print("Running with protoBuffer:   %s" % protoBuffer)
print("Running with quantInfo:     %s" % quantInfo)
print("Running with inputNode:     %s" % inputNode)
print("Running with outputNode:    %s" % outputNode)
print("Running with inputShape:    %s" % inputShape)
print("Running with means:         %s" % means)

### Step 3. Run the Quantizer (TBD)

Inspect the model to gather its input and output node(s), and input nodes' shape.  Next, quantize the model using graph and sample data parameters.  This quantization process produces two protobuf files containing the quantization information.  Finally, extract the quantization information using the compiler.  The end result is a txt file holding the scaling parameters for quantizing floats to INT8.  This is required because FPGAs will take advantage of Fixed Point Precision to achieve accelerated inference.  Remember to conda activate decent before beginning this section and conda deactivate to return to the ml-suite conda environment before moving on to the next section.

In [ ]:
calib_batch_size = 1

def calib_input(iter):
  images = []
  line = open(IMAGELIST).readlines()
  for index in range(0, calib_batch_size):
    curline = line[iter * calib_batch_size + index].strip()
    [calib_image_name, calib_label_id] = curline.split(' ')

    width_out = 128
    height_out = 224

    image = cv2.imread(IMAGEDIR + calib_image_name)
    width_in = image.shape[1]
    height_in = image.shape[0]

    scale = max(float(height_out) / height_in, float(width_out) / width_in)

    image = cv2.resize(image, (round(scale * width_in), round(s * height_in)))
    width_in = image.shape[1]
    height_in = image.shape[0]

    width_begin = round(0.5 * (width_in - width_out))
    height_begin = round(0.5 * (height_in - height_out))
    width_end = width_begin + width_out
    height_end = height_begin + height_out
    image = image[width_begin:width_end, height_begin:height_end]

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    means = (123, 117, 104)
    image -= means

    images.append(image)
  return {"data": images}


In [ ]:
!decent_q inspect --input_frozen_graph $protoBuffer

In [ ]:
!decent_q quantize \
    --input_frozen_graph $protoBuffer \
    --input_nodes $inputNode \
    --output_nodes $outputNode \
    --input_shapes ?,$inputShape,3 \
    --input_fn default \
    --method 1 \
    --calib_iter 1 \
    --batch_size 10 \
    --output_dir $MODELDIR \
    --image_dir $IMAGEDIR \
    --image_list $IMAGELIST \
    --means $means

In [ ]:
!python -m xfdnn.tools.compile.bin.xfdnn_compiler_tensorflow \
    -n $MODELDIR/deploy_model.pb \
    -m 9 \
    --dsp 96 \
    -g $quantInfo

### Step 4: Run the Partitioner and Compiler

The partitioner takes in the model protoBuffer, and pre-computer quantization parameters and compiles the porttion of the network specified from starnode(s) to finalnode(s) for FPGA acceleration.

In case startnode and/or finalnode is not specified (i.e., an empty list) the corresponding endnode is infered from the protoBuffer.

In [ ]:
def get_args(startnode=inputNode, finalnode=outputNode):
    return {
        ### Some standard partitioner arguments [EDITABLE]
        'startnode':            startnode,
        'finalnode':            finalnode,
        
        ### Some standard compiler arguments [PLEASE DONT TOUCH]
        'dsp':                  96,
        'memory':               9,
        'bytesperpixels':       1,
        'ddr':                  256,
        'data_format':          'NHWC',
        'mixmemorystrategy':    True,
        'noreplication':        True,
        'xdnnv3':               True,
        'usedeephi':            True,
        'quantz':               ''  
    }

In [ ]:
## load default arguments
FLAGS, unparsed = default_xdnn_arg_parser().parse_known_args([])

### Partition and compile
rt = xdnnRT(FLAGS,
            networkfile=protoBuffer,
            quant_cfgfile=quantInfo,
            xclbin=XCLBIN,
            device='FPGA',
            **get_args(inputNode, outputNode)
           )

### Step 5: Inference 

If the model in protoBuffer includes all the pre and post processings, the inference can be done simply by passing the input_data as follows:


In [ ]:
## Pre-processing function
def preprocess(image):
    input_height, input_width = 224, 224

    ## Image preprocessing using numpy
    img  = cv2.imread(image).astype(np.float32)
    img -= np.array(make_list(means)).reshape(-1,3).astype(np.float32)
    img  = cv2.resize(img, (input_width, input_height))
    
    return img

In [ ]:
## Choose image to run, display it for reference
image  = IMAGEDIR + "ILSVRC2012_val_00000003.JPEG"

display(filename=image)

In [ ]:
## Accelerated execution

## load the accelerated graph
graph = rt.load_partitioned_graph()

## run the tensorflow graph as usual (additional operations can be added to the graph)
with tf.Session(graph=graph) as sess:
    input_tensor  = graph.get_operation_by_name(inputNode).outputs[0]
    output_tensor = graph.get_operation_by_name(outputNode).outputs[0]
    
    predictions = sess.run(output_tensor, feed_dict={input_tensor: [preprocess(image)]})

In [ ]:
labels = np.loadtxt(LABELSLIST, str, delimiter='\t')
top_k = predictions[0].argsort()[:-6:-1]

for l,p in zip(labels[top_k], predictions[0][top_k]):
    print (l," : ",p)

# Conclusion
This notebook demonstrates how to target Xilinx FPGAs for inference using Caffe.  
The custom Python layer is defined in /opt/ml-suite/xfdnn/rt/scripts/framework/caffe/CaffeXFDNN.py  
When the time comes to take your application to production please look at examples in /opt/ml-suite/examples/deployment_modes/  
Highest performance is acheived by creating multiprocess pipelines.